## Primer challenge
#### Ce notebook fourni un cadre pour tester la specificite et la sensibilité d'amorces pour le qPCR


Ce cadre utilise le programme de recherche par similarité NCBI-blast+ et des bases de données du NCBI

In [1]:
# Importation des modules standards
import os
import glob
import re
from pathlib import Path

# Importation des modules de biopython
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIXML
from Bio.Blast.Applications import NcbiblastpCommandline
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Alphabet import IUPAC
from Bio import Entrez

#### Functions used in this notebook

In [2]:
def get_files(start_dir, file_type):
    """ Given a directory, make a list of files
    This function will find all files recursively.
    """
    files = []
    for filename in Path(start_dir).glob('**/*.' + file_type):
        files.append(str(filename))
    return files

In [3]:
def get_accession(string):
    """ Return the plain NCBI accession number
    e.g.: NC_123456, or NZ_123456
    """
    p = re.compile(r'.*\|ref\|([^\|]+)\|')
    m = p.match(string)
    if m:
        return(m.group(1))

In [4]:
def describe_amplicon(amplicon, cds_gene, cds_product, lower_pos, upper_pos):
    "Just nicely place infos about the amplicon generated by the two primers"
    offset=10
    print ('\n' + ' '*offset + 'amplicon_size:' + repr(amplicon) + ' bp')
    print (' '*offset + 'gene encompassing this region:' + cds_gene)
    print (' '*offset + 'product:' + cds_product + '\n')
    return

In [5]:
def place_revseq_primer(amplicon, primer, match, r_sbjct_start):
    """ Will place the reverse complement sequence of the given primer
    at the end of the amplicon sequence.
    """
    primer_len = len(primer)
    
    if len (amplicon) > 75:
        amplicon = amplicon[(len(amplicon)-75):]
    primer1 = Seq(primer, IUPAC.unambiguous_dna)    # turn f_query to a Seq object
    printer = ' '* (len(amplicon) - primer_len)
    printer2 = ' '* (len(amplicon) - len(repr(r_sbjct_start)))
    printer3 = ' '* (len(amplicon) -1)
    print (printer + '   ' + primer1.reverse_complement())
    print (printer + '   ' + match)
    print ('...' + amplicon)
    print (printer3 + '   |')
    print (printer2 + '   ' + repr(r_sbjct_start) + '\n')
    return

In [6]:
def place_forward_primer(amplicon, primer, match, f_sbjct_start):
    """ Will place the forward equence of the given primer
    at the beginning of the amplicon sequence. Easy!
    """
    print('\n' + primer)
    print(match)
    print(amplicon[:75] + '...')
    print('|\n' + repr(f_sbjct_start))
    return

In [7]:
def get_gb_alias(string):
    """ Return a NCBI genbank id with annotated features
    Sometimes NCBI ids poins to Genbank files that do not contain
    any annotated features, but have a link to another ID.
    The goal of this function is to get this ID.
    For example, we are looking for U00096.3 on the next line:
    join(U00096.3:1..4641652)
    """
    p = re.compile(r'join\(([^:]+)')
    m = p.match(string)
    if m:
        return(m.group(1))    

In [8]:
def get_gi(string):
    """ Return the NCBI gi identifiant number with the two pipes
    e.g.: gi|1231323|
    """
    p = re.compile(r'.*(gi\|\d+\|)')
    m = p.match(string)
    if m:
        return(m.group(1))

#### Global variables

In [9]:
Entrez.email = "jean-simon.brouard@canada.ca" # Tells ncbi who you are.

### Performing blastn searches against NCBI databases (e.g. ref_prok_rep_genomes)

### Note that NCBI databases are installed here /data/ext4/dataDP/db/NCBI on QCSHERA673892DX

In [10]:
# Create xml folder if it not exists
cwd = os.getcwd()
if not os.path.exists(cwd + '/xml'):
    os.mkdir(cwd + '/xml')

    
# First grab the query files in the queries folder
amorces_f = Path("./amorces/forward.fa")
amorces_r = Path("./amorces/reverse.fa")


# Get basenames
outname_f =  os.path.basename(amorces_f).replace('.fa','')
outname_r =  os.path.basename(amorces_r).replace('.fa','')


# Prepare the blastn command line with the forward primers
blastn_forward_cline = NcbiblastnCommandline(query = amorces_f,
                                             task = "megablast",
                                             word_size = 7,
                                             db = "ref_prok_rep_genomes",
                                             num_threads = 48,
                                             outfmt = 5,
                                             out = 'xml/' + outname_f + '.xml')
# Lauch the blastn command
stdout, stderr = blastn_forward_cline()


# Prepare the blastn command line with the reverse primers
blastn_reverse_cline = NcbiblastnCommandline(query = amorces_r,
                                             task = "megablast",
                                             word_size = 7,
                                             db = "ref_prok_rep_genomes",
                                             num_threads = 48,
                                             outfmt = 5,
                                             out='xml/' + outname_r + '.xml')
# Lauch the blastn command
stdout, stderr = blastn_reverse_cline()

# Our blast results are now in 2 xml files : forward.xml and reverse.xml

### First analysis of blast output (xml files)

Here we want to retain blastn alignments when both the forward and reverse primers have hits on the same molecule

In [11]:
# Define some lists
L1 = []
L2 = []
query_names = []
double_match = {}

# First grab the bastn output files in the xml folder
xml_forward = Path("./xml/forward.xml")
xml_reverse = Path("./xml/reverse.xml")

#p = re.compile(r'gi\|(\d+)\|')
S_reverse = set()

# Parsing of forward blast results
print ("*** Analyzing", outname_f, sep = ' ')
result_handle = open(xml_forward)
blast_forward_records = NCBIXML.parse(result_handle)
for blast_record_i, blast_record in enumerate(blast_forward_records, start=1):
    query_names.append(blast_record.query)    # Rember the query names
    S_forward = set()        # we have one forward set for each blast_record    
    print('* Query #' + repr(blast_record_i) + ' is ' + blast_record.query + '*', end='')
    print('(length=' + repr(blast_record.query_letters) + ') ', end='\n')
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            m = get_gi(alignment.title)
            if m:
                S_forward.add(m) # Add the GI|1323| id to a set    
    L1.append(S_forward)    # Remember the forward set for this blast record

# Parsing of reverse blast results
print ("*** Analyzing", outname_r, sep=' ')
result_handle = open(xml_reverse)
blast_reverse_records = NCBIXML.parse(result_handle)
for blast_record_i, blast_record in enumerate(blast_reverse_records, start=1):
    S_reverse = set()    # we have one reverse set for each blast_record
    print('* Query #' + repr(blast_record_i) + ' is ' + blast_record.query + '*', end='')
    print('(length=' + repr(blast_record.query_letters) + ') ', end='\n')    
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps: 
                m = get_gi(alignment.title)
                if m:
                    S_reverse.add(m) # Add the GI|1323| id to a set
    L2.append(S_reverse)    # Remember the forward set for this blast record
    
print ('Analyzing sets - intersection')

for name, l1, l2 in zip(query_names, L1, L2):
    print (name, l1 & l2) # the result of intersection between gi ids of forward and reverse sets
    double_match[name]= (l1 & l2)
    
print (double_match)

*** Analyzing forward
* Query #1 is amorce1*(length=18) 
* Query #2 is amorce2*(length=19) 
* Query #3 is amorce3*(length=20) 
* Query #4 is amorce4*(length=20) 
* Query #5 is amorce5*(length=18) 
* Query #6 is amorce6*(length=20) 
* Query #7 is amorce7*(length=20) 
* Query #8 is amorce8*(length=20) 
* Query #9 is amorce9*(length=20) 
* Query #10 is amorce10*(length=21) 
* Query #11 is amorce11*(length=20) 
* Query #12 is amorce12*(length=21) 
* Query #13 is amorce13*(length=21) 
* Query #14 is amorce14*(length=20) 
* Query #15 is amorce15*(length=21) 
* Query #16 is amorce16*(length=20) 
*** Analyzing reverse
* Query #1 is amorce1*(length=18) 
* Query #2 is amorce2*(length=18) 
* Query #3 is amorce3*(length=18) 
* Query #4 is amorce4*(length=18) 
* Query #5 is amorce5*(length=18) 
* Query #6 is amorce6*(length=20) 
* Query #7 is amorce7*(length=19) 
* Query #8 is amorce8*(length=17) 
* Query #9 is amorce9*(length=20) 
* Query #10 is amorce10*(length=19) 
* Query #11 is amorce11*(lengt

### 2nd analysis of blast output (xml files)

##### This time, we will grab some alignments objects in a dictionnary

In [12]:
# We iterate a 2nd time on our xml files, but this time, we will grab some alignments objects

d = {}

print ("*** Keeping double match alignment objects for", outname_f, "primers ***", sep = ' ')
result_handle = open(xml_forward)
blast_forward_records = NCBIXML.parse(result_handle)
for blast_record_i, blast_record in enumerate(blast_forward_records):
    for alignment in blast_record.alignments:
        # Keeping alignements in a dictionnary if they have been recorded as double match
        gi = get_gi(alignment.title)
        if gi in double_match[blast_record.query]:
            # Check the existence of a entry named blast_record.query in d 
            if not blast_record.query in d:
                d[blast_record.query] = {} # Create the main key if it does not exists
            
            if not gi in d[blast_record.query]:
                d[blast_record.query][gi] = {} # Create the second key
                
            d[blast_record.query][gi]['aln_f'] = alignment
            
            # Add the genbank id
            accession = get_accession(alignment.title)
            d[blast_record.query][gi]['accession'] = accession
            
print ("*** Keeping double match alignment objects for", outname_r, "primers ***", sep = ' ')
result_handle = open(xml_reverse)
blast_reverse_records = NCBIXML.parse(result_handle)
for blast_record_i, blast_record in enumerate(blast_reverse_records):
    for alignment in blast_record.alignments:
        # Keeping alignements in a dictionnary if they have been recorded as double match
        gi = get_gi(alignment.title)
        if gi in double_match[blast_record.query]:
            # Check the existence of a entry named blast_record.query in d 
            if not blast_record.query in d:
                d[blast_record.query] = {} # Create the main key if it does not exists
            
            if not gi in d[blast_record.query]:
                d[blast_record.query][gi] = {} # Create the second key
                
            d[blast_record.query][gi]['aln_r'] = alignment
            
            # Add the genbank id
            accession = get_accession(alignment.title)
            d[blast_record.query][gi]['accession'] = accession

*** Keeping double match alignment objects for forward primers ***
*** Keeping double match alignment objects for reverse primers ***


In [13]:
for key, value in d.items():
    print ('_'*78 + '\n')
    print('Blastn (Megablast) matchs for primer pair:', key)
    print ('_'*78 + '\n')

    for sub_key, sub_value in value.items():
        print (">>>", sub_value['aln_f'].title)
    
        # Get infos from our dict d
        f_query = sub_value['aln_f'].hsps[0].query
        f_sbjct = sub_value['aln_f'].hsps[0].sbjct
        f_match = sub_value['aln_f'].hsps[0].match
        f_sbjct_start = sub_value['aln_f'].hsps[0].sbjct_start
    
        r_query = sub_value['aln_r'].hsps[0].query
        r_sbjct = sub_value['aln_r'].hsps[0].sbjct
        r_match = sub_value['aln_r'].hsps[0].match
        r_sbjct_start = sub_value['aln_r'].hsps[0].sbjct_start
    
        # This snippet of code calculate the amplicon size.
        # The lower and upper position values come from different
        # alignements depending on if the 'forward primer' bind on the plus strand of the
        # reference sequence or not
    
        if r_sbjct_start > f_sbjct_start:
            lower_pos = f_sbjct_start -1
            upper_pos = r_sbjct_start -1
        else:
            lower_pos = r_sbjct_start -1
            upper_pos = f_sbjct_start -1
    
        amplicon = upper_pos - lower_pos + 1    # amplicon_size
        gb = sub_value['accession']
    
        # Esearch just check the existence of the entry
        handle = Entrez.esearch(db = "nucleotide", term = gb, rettype = "gb", retmode = "text")
        record = Entrez.read(handle)
        if len(record["IdList"]) == 0: # if no hit in genbank
            print ('no hit hit in genbank for', gb)

        with Entrez.efetch(
            db = "nucleotide", rettype="gb", retmode="text", id = gb) as handle:
            seq_record = SeqIO.read(handle, "gb")  # using "gb" as an alias for "genbank"
            # print("%s with %i features" % (seq_record.id, len(seq_record.features)))

            if (len(seq_record.features) == 1):
                # to see the keys: print(seq_record.annotations.keys())
                contig_str = seq_record.annotations["contig"]
                gb2 = get_gb_alias(contig_str)
                # print ('Swichting from', gb, 'to', gb2)
                gb = gb2
                with Entrez.efetch(db = "nucleotide", rettype="gb", retmode="text", id = gb) as handle:
                    seq_record = SeqIO.read(handle, "gb")  # using "gb" as an alias for "genbank"
                    # print("%s with %i features" % (seq_record.id, len(seq_record.features)))


        # From Bio import SeqIO 4.3.2.4  Location testing
        # Here is a simple brute force solution where we just check all the features one by one in a loop:
        for feature in seq_record.features[1:]:
            if lower_pos in feature:
                if feature.type == 'CDS':
                    if 'gene' in feature.qualifiers:
                        #print("%s %s" % ('-->gene in CDS field:', feature.qualifiers['gene'][0]))
                        cds_gene = feature.qualifiers['gene'][0]
                    if 'product' in feature.qualifiers:
                        #print("%s %s" % ('-->product in CDS field:', feature.qualifiers['product'][0]))
                        cds_product = feature.qualifiers['product'][0]
    
        # Working with our sub_record
        sub_record = seq_record[lower_pos:upper_pos + 1] # String slices[start_inclusive:stop_exclusive]
        amplicon_seq = str(sub_record.seq)
    
        # Here we try do depict the binding of the primers on the amplicon
        # Normal case
        if r_sbjct_start > f_sbjct_start:
            place_forward_primer(amplicon_seq, f_query, f_match, f_sbjct_start)
            describe_amplicon(amplicon, cds_gene, cds_product, lower_pos, upper_pos)
            place_revseq_primer(amplicon_seq, r_query, r_match, r_sbjct_start)
    
        # Case when the 'forward' primerbind to the reverse complement
        # strand of the reference genome
        else:
            place_forward_primer(amplicon_seq, r_query, r_match, f_sbjct_start)
            describe_amplicon(amplicon, cds_gene, cds_product, lower_pos, upper_pos)
            place_revseq_primer(amplicon_seq, f_query, f_match, r_sbjct_start)
          

______________________________________________________________________________

Blastn (Megablast) matchs for primer pair: amorce7
______________________________________________________________________________

>>> gi|218703261|ref|NC_011751.1| Escherichia coli UMN026, complete genome

TCGATGCCGACAATGACATC
||||||||||||||||||||
TCGATGCCGACAATGACATCCGTGCCCAGGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCC...
|
4039495

          amplicon_size:63 bp
          gene encompassing this region:zntA
          product:zinc, cobalt and lead efflux system

                                               CAAAAAGCGGGCTATTCCC
                                               |||||||||||||||||||
...TCGATGCCGACAATGACATCCGTGCCCAGGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCC
                                                                 |
                                                           4039557

>>> gi|407479587|ref|NC_018658.1| Escherichia coli O104:H4 str. 2011C-3493 chromosome, complete genome

GGGAATAGCCCGCTTTTTG
|||


GATGCCGACAATGACATC
||||||||||||||||||
GATGCCGACAATGACATCCGTGCACAAGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCCT...
|
4326096

          amplicon_size:62 bp
          gene encompassing this region:zntA
          product:Cd2+/Zn2+-exporting ATPase

                                             CAAAAAGCGGGCTATTCCCT
                                             ||||||||||||||||||||
...GATGCCGACAATGACATCCGTGCACAAGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCCT
                                                                |
                                                          4326157

>>> gi|387615344|ref|NC_017634.1| Escherichia coli O83:H1 str. NRG 857C chromosome, complete genome

GATGCCGACAATGACATC
||||||||||||||||||
GATGCCGACAATGACATCCGGGCACAAGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCCT...
|
3626895

          amplicon_size:62 bp
          gene encompassing this region:zntA
          product:zinc/cadmium/mercury/lead-transporting ATPase

                                             CAAAAAGCGGGCTATTCCCT
          


GATGCCGACAATGACATC
||||||||||||||||||
GATGCCGACAATGACATCCGTGCCCAGGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCCTG...
|
4107713

          amplicon_size:63 bp
          gene encompassing this region:zntA
          product:zinc, cobalt and lead efflux system

                                              AAAAAGCGGGCTATTCCCTG
                                              ||||||||||||||||||||
...GATGCCGACAATGACATCCGTGCCCAGGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCCTG
                                                                 |
                                                           4107775

>>> gi|82775382|ref|NC_007606.1| Shigella dysenteriae Sd197 chromosome, complete genome

GATGCCGACAATGACATC
||||||||||||||||||
GATGCCGACAATGACATCCGTGCCCAGGTTGAATCTGCGGTGCAAAAAGCGGGCTATTCCCT...
|
3355409

          amplicon_size:62 bp
          gene encompassing this region:zntA
          product:zinc-transporting ATPase

                                              AAAAAGCGGGCTATTCCCT
                            